In [ ]:
import numpy as np

In [ ]:
GRID_SIZE = 3 * 3
NB_INDIVIDUALS_POPULATION = 50

In [ ]:
def generate_individual():
    """
    Initialize a random Sudoku grid where each cell has an equal chance of being a number from 1 to 9.
    So the grid can be invalid. But aftern, we will use a genetic to make the grid valid.

    Returns:
        A numpy.ndarray with dimensions (GRID_SIZE, GRID_SIZE) where each cell contains a value between 1 and GRID_SIZE (inclusive).
    """
    min_value = 1
    max_value = GRID_SIZE + 1
    size = (GRID_SIZE, GRID_SIZE)
    return np.random.randint(min_value, max_value, size=size)


In [ ]:
generate_individual()

In [ ]:
def generate_population():
    """
    Generates a population by creating a list of individuals using the generate_individual function.

    Returns:
        list: A population consisting of multiple individuals.
    """
    return [generate_individual() for x in range(NB_INDIVIDUALS_POPULATION)]

In [ ]:
def compute_fitness(individual):
    """
    Computes the fitness score of an individual.

    Args:
        individual (numpy.ndarray): A numpy.ndarray with dimensions (GRID_SIZE, GRID_SIZE) where each cell contains a value between 1 and GRID_SIZE (inclusive).

    Returns:
        int: Fitness score of the individual.
    """
    fitness = 0

    for i in range(GRID_SIZE):
        unique_digit_column = np.unique(individual[:, i])
        unique_digit_line = np.unique(individual[i])
        fitness += len(unique_digit_line)
        fitness += len(unique_digit_column)
        
    return fitness